## Protocol 4 : Synchronization Analysis Between Triggers and Visual Stimuli

This protocol aims to measure the delay and jitter between a trigger sent by an Arduino board and the actual appearance of a visual stimulus on a screen.

### Prerequisites

**Hardware:**
* All the hardware from Protocols 1 and 2

**Software:**
* All the software and libraries from Protocols 1 and 2

### Procedure

#### 1. Generating Triggers and Stimuli

1. **Use two screens**: one for the acquisition window (Waveforms) and one for the visual stimuli display.

2. **Prepare the `.ino` and `.py` code**. Together, they generate triggers on a pin (here pin 8) at each stimulus display, using a defined period.

> **Arduino (.ino) code to receive serial commands**
> ```arduino
> #include <TimerOne.h>
> 
> const int triggerPin = 8;
> volatile bool doit_envoyer = false;
> 
> void setup() {
>   Serial.begin(115200);
>   pinMode(triggerPin, OUTPUT);
>   digitalWrite(triggerPin, LOW);  
>   Timer1.initialize(500000);  // 500 ms
>   Timer1.attachInterrupt(routine_periode);
> }
> 
> void loop() {
>   if (doit_envoyer) {
>     doit_envoyer = false;
>     Serial.println("SHOW");
>     Serial.flush();
>     digitalWrite(triggerPin, HIGH);  
>     delayMicroseconds(1000);
>     digitalWrite(triggerPin, LOW); 
>   }
> }
> 
> void routine_periode() {
>   doit_envoyer = true;
> }
> ```

3. **Upload the `.ino` code** to your Arduino board following steps 2–5 of Protocol 1.

4. **Prepare your Python (.py) script** that displays the stimulus and listens for the "SHOW" command from the Arduino.

> **Python (.py) code to display stimuli and synchronize with triggers**
> ```python
> from expyriment import design, control, stimuli, misc
> import serial
> 
> SERIAL_PORT = '/dev/ttyACM0'  # Adjust as needed
> BAUDRATE = 115200
> SQUARE_DURATION = 100  # ms
> 
> arduino = serial.Serial(SERIAL_PORT, BAUDRATE, timeout=0.1)
> 
> exp = design.Experiment(name="Square Display on Serial Trigger")
> control.set_develop_mode(True)
> control.initialize(exp)
> 
> square = stimuli.Rectangle((400, 400))
> blank = stimuli.BlankScreen()
> square.preload()
> blank.preload()
> 
> control.start(skip_ready_screen=True)
> 
> clock = misc.Clock()
> print("✅ Ready. Waiting for 'SHOW' serial commands...")
> 
> try:
>     while True:
>         if arduino.in_waiting:
>             line = arduino.readline().decode(errors="ignore").strip()
>             if line == "SHOW":
>                 square.present(update=True)
>                 clock.wait(SQUARE_DURATION)
>                 blank.present(update=True)
>         exp.keyboard.process_control_keys()
> except KeyboardInterrupt:
>     print("Manually interrupted.")
> finally:
>     arduino.close()
>     control.end()
> ```

5. **Run the `.py` script** in the terminal.  
> ![Scope settings](../pictures/arduino_trigger_master_timer.png)

6. A window should now display stimuli on the secondary screen. The Arduino should send a trigger each time.

---

#### 2. Signal Acquisition

1. **Connect the oscilloscope probes:**

   * **Channel 1 (Arduino Trigger):**
     * GND to Arduino's ground
     * Probe to **pin 8** of the Arduino

   * **Channel 2 (Photodiode):**
     * GND to the photodiode's ground
     * Probe to photodiode's Vout

2. **Place the photodiode** in front of the stimulus display window.

3. Start acquisition by clicking **`Run`**. You should see two signals:
   * **Channel 1:** TTL trigger from the Arduino
   * **Channel 2:** Light-dependent photodiode signal  
   Both signals should have the same period, but possibly different duty cycles.  
> ![Scope settings](../pictures/acquisition_arduino_photodiode.png)

4. **Adjust the axis scales**  
> ![Scope settings](../pictures/acquisition_characteristics_two_channels.png)

---

#### 3. Recording the Signals

1. Follow the same recording procedure as in Protocols 1 and 2, ensuring both channels are recorded. You will obtain a multi-column `.csv` file.  
> ![Scope settings](../pictures/record_button.png)  
> ![Scope settings](../pictures/choice_of_recording_characteristics_two_channels_wav.png)

---

#### 4. Signal Analysis

1. **Prepare your Python script** to analyze a two-channel `.wav` file. It should detect peaks in both channels and calculate the delay between Arduino triggers and photodiode peaks.

> **Python (.py) script for synchronization analysis**
> ```python
> import numpy as np
> import matplotlib
> matplotlib.use('Agg')
> import matplotlib.pyplot as plt
> from scipy.io import wavfile
> from scipy.signal import find_peaks
> import os
> 
> NOM_FICHIER_WAV = 'arduino_trigger_master_timer_500ms_both_2.wav'
> DISTANCE_MIN_MS = 100.0
> 
> def analyser_signaux_wav(nom_fichier):
>     sampling_rate, data = wavfile.read(nom_fichier)
>     if data.ndim != 2 or data.shape[1] < 2:
>         raise ValueError("WAV must have at least 2 channels (Arduino, Photodiode)")
> 
>     ttl = data[:, 0]
>     photodiode = data[:, 1]
> 
>     print(f"--- File: {nom_fichier} ---")
>     print(f"Sampling rate: {sampling_rate} Hz")
> 
>     seuil_ttl = 0.5 * np.max(ttl)
>     signal_photo_abs = np.abs(photodiode)
>     seuil_photo = 0.8 * np.max(signal_photo_abs)
>     print(f"TTL threshold: {seuil_ttl:.2f}, Photodiode threshold: {seuil_photo:.2f}")
> 
>     min_dist_samples = int((DISTANCE_MIN_MS / 1000) * sampling_rate)
>     ttl_peaks, _ = find_peaks(ttl, height=seuil_ttl, distance=min_dist_samples)
>     photo_peaks, _ = find_peaks(signal_photo_abs, height=seuil_photo, distance=min_dist_samples)
> 
>     def analyse_intervalles(peaks, label):
>         intervals = np.diff(peaks) / sampling_rate * 1000
>         mean = np.mean(intervals)
>         print(f"\n--- {label} ---")
>         print(f"Triggers: {len(peaks)}")
>         print(f"Mean: {mean:.2f} ms")
>         print(f"Min: {np.min(intervals):.2f}, Max: {np.max(intervals):.2f}, Std: {np.std(intervals):.2f}")
>         plt.figure()
>         plt.hist(intervals, bins=40, edgecolor='black', alpha=0.8)
>         plt.axvline(mean, color='red', linestyle='--', label=f'Mean = {mean:.2f} ms')
>         plt.title(f'Intervals – {label}')
>         plt.xlabel("Duration (ms)")
>         plt.ylabel("Count")
>         plt.legend()
>         os.makedirs("figures", exist_ok=True)
>         plt.savefig(f"figures/hist_intervals_{label.lower().replace(' ', '_')}.png")
>         plt.close()
> 
>     analyse_intervalles(ttl_peaks, "Arduino")
>     analyse_intervalles(photo_peaks, "Photodiode")
> 
>     matched_delays = []
>     for t in ttl_peaks:
>         deltas = photo_peaks - t
>         pos_deltas = deltas[deltas > 0]
>         if len(pos_deltas):
>             delay = pos_deltas[0] / sampling_rate * 1000
>             matched_delays.append(delay)
> 
>     matched_delays = np.array(matched_delays)
>     print("\n--- Arduino → Photodiode Delay ---")
>     print(f"Matches: {len(matched_delays)}")
>     print(f"Mean: {np.mean(matched_delays):.2f} ms, Min: {np.min(matched_delays):.2f}, Max: {np.max(matched_delays):.2f}, Jitter: {np.std(matched_delays):.2f} ms")
>     plt.figure()
>     plt.hist(matched_delays, bins=30, edgecolor='black', alpha=0.8)
>     plt.axvline(np.mean(matched_delays), color='red', linestyle='--', label=f'Mean = {np.mean(matched_delays):.2f} ms')
>     plt.title("Delay Between Arduino Trigger and Visual Detection")
>     plt.xlabel("Delay (ms)")
>     plt.ylabel("Occurrences")
>     plt.legend()
>     plt.savefig("figures/hist_delay_trigger_to_photo.png")
>     plt.close()
> ```

2. **Run the script.**  
> ![Scope settings](../pictures/analyse_and_draw_triggers_photodiode.png)

3. The terminal will show the characteristics of both signals and delay intervals. You will also find saved histograms in the specified folder.  
> ![Scope settings](../pictures/command_results_arduino_photodiode.png)  
> ![Scope settings](../pictures/results_plot_arduino_photodiode2.png)  
> ![Scope settings](../pictures/results_plot_photodiode_both.png)  
> ![Scope settings](../pictures/results_plot_arduino_both.png)